In [ ]:
import os
import logging
import paho.mqtt.client as mqtt

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s'
)

FLEET = os.getenv("FLEET", "lab")
MQTT_BROKER = os.getenv("MQTT_BROKER", "localhost")
MQTT_PORT = int(os.getenv("MQTT_PORT", 1883))

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        logging.info("MQTT 브로커 연결 성공")
        
        topic = f"drone/{FLEET}/+/telemetry/#"
        client.subscribe(topic)
        logging.info(f"토픽 구독: {topic}")
    else:
        logging.error(f"연결 실패, 코드: {rc}")

def on_message(client, userdata, message):
    logging.info(f"메시지 수신 - Topic: {message.topic}")
    logging.info(f"Payload: {message.payload.decode('utf-8')}")

def on_disconnect(client, userdata, rc):
    logging.warning(f"연결 끊김, 코드: {rc}")

if __name__ == "__main__":
    client = mqtt.Client()
    
    client.on_connect = on_connect
    client.on_message = on_message
    client.on_disconnect = on_disconnect
    
    try:
        logging.info(f"브로커 연결 시도: {MQTT_BROKER}:{MQTT_PORT}")
        client.connect(MQTT_BROKER, MQTT_PORT, 60)
        
        logging.info("메시지 수신 대기 중... (Ctrl+C로 종료)")
        client.loop_forever()
        
    except KeyboardInterrupt:
        logging.info("프로그램 종료")
        client.disconnect()
    except Exception as e:
        logging.error(f"오류 발생: {e}")
        client.disconnect()